In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   PPL     NLL   
5000     68   35   99  519.71  810  516.70  2.88  0.13  973.64  1457  967.15   
10000    61   71   59  518.15  793  514.60  3.44  0.11  959.52  1350  952.62   
15000    65  107   19  517.93  788  513.98  3.85  0.10  943.11  1159  935.54   
20000    59  142  119  517.23  783  513.05  4.09  0.09  941.49  1165  934.19   
25000    56  178   79  516.73  778  512.41  4.24  0.08  945.01  1182  938.11   
30000    60  214   39  516.34  773  511.91  4.35  0.07  945.45  1177  938.58   
35000    56  249  139  516.05  770  511.56  4.43  0.06  942.67  1163  935.91   
40000    61  285   99  515.60  766  511.04  4.50  0.06  944.85  1178  938.02   
45000    57  321   59  515.41  764  510.80  4.55  0.05  945.68  1181  938.98   
50000    60  357   19  515.22  761  510.57  4.60  0.05  944.74  1167  937.86   
55000    56  392  119  515.23  762  510.59  4.59  0.05  947.70  1205  941.21   
60000    72  428   79  515.20  762  510.56  4.59  0.05  946.11  1185  939.69   
65000    81  464   39  515.20  761  510.56  4.59  0.05  947.65  1191  941.15   
70000    76  499  139  515.17  761  510.52  4.60  0.04  945.39  1180  938.94   
75000    54  535   99  515.06  760  510.41  4.60  0.04  945.89  1183  939.49   
80000    55  571   59  515.01  759  510.37  4.60  0.04  947.90  1198  941.50   
85000    50  607   19  515.31  761  510.69  4.58  0.04  945.93  1189  940.31   
90000    52  642  119  515.50  763  510.90  4.55  0.04  949.33  1226  943.74   
95000    74  678   79  515.66  765  511.09  4.52  0.04  948.97  1225  943.49   
100000  145  714   39  515.69  765  511.14  4.51  0.04  947.31  1195  941.09   
105000  123  749  139  515.87  766  511.33  4.49  0.04  945.06  1187  939.46   
110000  139  785   99  515.80  766  511.28  4.48  0.04  947.06  1202  940.94   
115000  131  821   59  515.76  765  511.25  4.48  0.04  947.58  1201  941.42   
120000   75  857   19  515.92  766  511.42  4.46  0.04  946.90  1195  941.25   
125000  106  892  119  515.86  766  511.36  4.46  0.04  947.84  1209  941.74   
130000  124  928   79  515.79  766  511.30  4.46  0.04  946.94  1203  940.86   
135000  134  964   39  515.77  765  511.27  4.46  0.04  947.30  1195  941.13   
140000  122  999  139  515.91  766  511.42  4.44  0.04  946.27  1195  940.65   

                     TEST:       SPEC:             HIER:        
          KL   REG    LOSS   PPL     1     2     3 CHILD OTHER  
5000    6.32  0.17  601.48  1148  0.30  0.38  0.57  0.45  0.27  
10000   6.84  0.06  593.48  1048  0.38  0.37  0.49  0.38  0.29  
15000   7.52  0.06  578.16   888  0.32  0.29  0.43  0.42  0.36  
20000   7.25  0.05  578.16   888  0.37  0.41  0.46  0.38  0.28  
25000   6.87  0.02  578.16   888  0.35  0.38  0.42  0.40  0.32  
30000   6.84  0.03  578.16   888  0.29  0.32  0.51  0.40  0.30  
35000   6.74  0.02  578.16   888  0.29  0.38  0.48  0.37  0.28  
40000   6.80  0.03  578.16   888  0.34  0.39  0.52  0.36  0.26  
45000   6.68  0.02  578.16   888  0.36  0.34  0.46  0.41  0.31  
50000   6.86  0.03  578.16   888  0.31  0.37  0.52  0.38  0.27  
55000   6.47  0.03  578.16   888  0.28  0.38  0.51  0.39  0.25  
60000   6.40  0.02  578.16   888  0.33  0.37  0.50  0.40  0.25  
65000   6.48  0.02  578.16   888  0.28  0.31  0.50  0.42  0.28  
70000   6.44  0.02  578.16   888  0.29  0.35  0.50  0.40  0.28  
75000   6.37  0.02  578.16   888  0.33  0.36  0.50  0.39  0.27  
80000   6.38  0.02  578.16   888  0.34  0.31  0.48  0.43  0.31  
85000   5.60  0.02  578.16   888  0.26  0.27  0.45  0.48  0.33  
90000   5.57  0.03  578.16   888  0.25  0.32  0.35  0.46  0.37  
95000   5.46  0.02  578.16   888  0.29  0.31  0.34  0.47  0.39  
100000  6.19  0.04  578.16   888  0.26  0.30  0.39  0.47  0.37  
105000  5.58  0.02  578.16   888  0.25  0.29  0.36  0.47  0.38  
110000  6.09  0.04  578.16   888  0.31  0.31  0.39  0.46  0.37  
115000  6.14  0.03  578.16   888  0.32  0.30  0.38  0.47  0.37

0 R: 1.000 P: 0.052 use file get write thanks program window know run work
   1 R: 0.327 P: 0.098 write article get go one like car look good know
     11 R: 0.208 P: 0.208 write article get one like think know people say see
     13 R: 0.021 P: 0.021 game team play player year hockey season win go get
   2 R: 0.620 P: 0.119 use key chip government encryption system one law phone need
     21 R: 0.468 P: 0.468 god say one people write think know make believe see
     22 R: 0.033 P: 0.033 space drive launch system use nasa satellite datum new disk
{0: [1, 2, 3], 1: [11, 12], 2: [21, 23], 3: [31]}


TRAIN:                           VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   PPL     NLL   
5000     68   35   99  519.71  810  516.70  2.88  0.13  973.64  1457  967.15   
10000    61   71   59  518.15  793  514.60  3.44  0.11  959.52  1350  952.62   
15000    65  107   19  517.93  788  513.98  3.85  0.10  943.11  1159  935.54   
20000    59  142  119  517.23  783  513.05  4.09  0.09  941.49  1165  934.19   
25000    56  178   79  516.73  778  512.41  4.24  0.08  945.01  1182  938.11   
30000    60  214   39  516.34  773  511.91  4.35  0.07  945.45  1177  938.58   
35000    56  249  139  516.05  770  511.56  4.43  0.06  942.67  1163  935.91   
40000    61  285   99  515.60  766  511.04  4.50  0.06  944.85  1178  938.02   
45000    57  321   59  515.41  764  510.80  4.55  0.05  945.68  1181  938.98   
50000    60  357   19  515.22  761  510.57  4.60  0.05  944.74  1167  937.86   
55000    56  392  119  515.23  762  510.59  4.59  0.05  947.70  1205  941.21   
60000    72  428   79  515.20  762  510.56  4.59  0.05  946.11  1185  939.69   
65000    81  464   39  515.20  761  510.56  4.59  0.05  947.65  1191  941.15   
70000    76  499  139  515.17  761  510.52  4.60  0.04  945.39  1180  938.94   
75000    54  535   99  515.06  760  510.41  4.60  0.04  945.89  1183  939.49   
80000    55  571   59  515.01  759  510.37  4.60  0.04  947.90  1198  941.50   
85000    50  607   19  515.31  761  510.69  4.58  0.04  945.93  1189  940.31   
90000    52  642  119  515.50  763  510.90  4.55  0.04  949.33  1226  943.74   
95000    74  678   79  515.66  765  511.09  4.52  0.04  948.97  1225  943.49   
100000  145  714   39  515.69  765  511.14  4.51  0.04  947.31  1195  941.09   
105000  123  749  139  515.87  766  511.33  4.49  0.04  945.06  1187  939.46   
110000  139  785   99  515.80  766  511.28  4.48  0.04  947.06  1202  940.94   
115000  131  821   59  515.76  765  511.25  4.48  0.04  947.58  1201  941.42   
120000   75  857   19  515.92  766  511.42  4.46  0.04  946.90  1195  941.25   
125000  106  892  119  515.86  766  511.36  4.46  0.04  947.84  1209  941.74   
130000  124  928   79  515.79  766  511.30  4.46  0.04  946.94  1203  940.86   
135000  134  964   39  515.77  765  511.27  4.46  0.04  947.30  1195  941.13   
140000  122  999  139  515.91  766  511.42  4.44  0.04  946.27  1195  940.65   

                     TEST:       SPEC:             HIER:        
          KL   REG    LOSS   PPL     1     2     3 CHILD OTHER  
5000    6.32  0.17  601.48  1148  0.30  0.38  0.57  0.45  0.27  
10000   6.84  0.06  593.48  1048  0.38  0.37  0.49  0.38  0.29  
15000   7.52  0.06  578.16   888  0.32  0.29  0.43  0.42  0.36  
20000   7.25  0.05  578.16   888  0.37  0.41  0.46  0.38  0.28  
25000   6.87  0.02  578.16   888  0.35  0.38  0.42  0.40  0.32  
30000   6.84  0.03  578.16   888  0.29  0.32  0.51  0.40  0.30  
35000   6.74  0.02  578.16   888  0.29  0.38  0.48  0.37  0.28  
40000   6.80  0.03  578.16   888  0.34  0.39  0.52  0.36  0.26  
45000   6.68  0.02  578.16   888  0.36  0.34  0.46  0.41  0.31  
50000   6.86  0.03  578.16   888  0.31  0.37  0.52  0.38  0.27  
55000   6.47  0.03  578.16   888  0.28  0.38  0.51  0.39  0.25  
60000   6.40  0.02  578.16   888  0.33  0.37  0.50  0.40  0.25  
65000   6.48  0.02  578.16   888  0.28  0.31  0.50  0.42  0.28  
70000   6.44  0.02  578.16   888  0.29  0.35  0.50  0.40  0.28  
75000   6.37  0.02  578.16   888  0.33  0.36  0.50  0.39  0.27  
80000   6.38  0.02  578.16   888  0.34  0.31  0.48  0.43  0.31  
85000   5.60  0.02  578.16   888  0.26  0.27  0.45  0.48  0.33  
90000   5.57  0.03  578.16   888  0.25  0.32  0.35  0.46  0.37  
95000   5.46  0.02  578.16   888  0.29  0.31  0.34  0.47  0.39  
100000  6.19  0.04  578.16   888  0.26  0.30  0.39  0.47  0.37  
105000  5.58  0.02  578.16   888  0.25  0.29  0.36  0.47  0.38  
110000  6.09  0.04  578.16   888  0.31  0.31  0.39  0.46  0.37  
115000  6.14  0.03  578.16   888  0.32  0.30  0.38  0.47  0.37

0 R: 1.000 P: 0.046 use file get write thanks program window know run work
   1 R: 0.330 P: 0.101 write article get go one like car look good know
     11 R: 0.206 P: 0.206 write article get one like think know people say see
     12 R: 0.023 P: 0.023 game team play player year hockey season win go get
   2 R: 0.604 P: 0.110 use key chip government encryption system one law phone need
     21 R: 0.463 P: 0.463 god say one people write think know make believe see
     23 R: 0.030 P: 0.030 space drive launch system use nasa satellite datum new disk
   3 R: 0.020 P: 0.010 use switch amp water wire audio power cable circuit one
     31 R: 0.010 P: 0.010 drive card apple mac monitor disk chip ram use modem
